In [4]:
import torch
import torchmetrics
from TorchTemplates import SimpleBinaryClassificationNN as sbcnn

In [7]:
sbcnn.layer_template()


                    {
                    'input': {'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None},
                    'fcn'  : {'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None},
                    'output':{'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None}
                    }
                    


In [6]:
model = SimpleBinaryClassificationNN(layer_dict={
    'input':('n':[100,10], 'batch_norm':True, 'dropout':0.5, 'activation':'ReLU'),
    'output':([10,1], True, 0.5, None),
},
l1_lambda=0.1,
l2_lambda=0.2)

SyntaxError: invalid syntax (3266276176.py, line 2)

In [107]:
def fn():
    return print("""
    {
    'input': {'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None},
    'fcn'  : {'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None},
    'output':{'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None}
    }
    """)

In [108]:
fn()


    {
    'input': {'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None},
    'fcn'  : {'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None},
    'output':{'units':[None,None], 'batch_norm': None, 'dropout': None, 'activation': None}
    }
    


In [103]:
fn()

{'input': {'units': [None, None],
  'batch_norm': None,
  'dropout': None,
  'activation': None},
 'fcn': {'units': [None, None],
  'batch_norm': None,
  'dropout': None,
  'activation': None},
 'output': {'units': [None, None],
  'batch_norm': None,
  'dropout': None,
  'activation': None}}

In [ ]:
layer_dict = {'input':{'units':[100,10], 'batch_norm':True, 'dropout': 0.5, 'activation':'ReLU'},
              'output':{'units':[10,1], 'batch_norm':True, 'dropout': 0.5, 'activation':None}}

In [50]:
(torch.tensor([1.,2.,3.],requires_grad=True) + 0).requires_grad

True

In [ ]:
torchmetrics.Binar

In [96]:
eval_func = getattr(torchmetrics, 'AUROC')(task='binary')
float(eval_func(torch.tensor([0.51,0.1, 0.2,0.1]), torch.tensor([1.,0.,0.,1.])))

0.625

In [81]:
eval_func()

TypeError: BinaryPrecisionRecallCurve.update() missing 2 required positional arguments: 'preds' and 'target'

In [78]:
eval_func.

True

In [66]:
import numpy as np
roc_auc_score(np.array([1,0,0,1]), np.array([0.5,0.1, 0.2,0.1]))

0.625